In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import getpass

In [2]:
op = webdriver.FirefoxOptions()
op.add_argument('--headless')
driver = webdriver.Firefox(executable_path=r"/home/zachlim/Documents/geckodriver", options=op)
driver.get("https://www.barchart.com/options/iv-rank-percentile/etfs?orderBy=optionsImpliedVolatilityPercentile1y&orderDir=desc")

source_page = driver.page_source

ticker_name = driver.find_element_by_xpath("/html/body/main/div/div[2]/div[2]/div/div[2]/div/div/div/div[6]/div/div[2]/div/div/ng-transclude/table")

In [11]:
class Renderer:
    from rich.console import Console
    from rich.table import Table

    def __init__(self):
        self.console = Console()
    
    def last_color(self, text):
        if float(text) < 30:
            return f"[bright_green]{str(text)}[/]"
        elif float(text) > 30 and float(text) < 50:
            return f"[orange1]{str(text)}[/]"
        else:
            return f"[bright_red]{str(text)}[/]"

    def screener_table(self, data, table_type="ETF"):
        table = Table(title=f"Screened {table_type}")
        col_name = ["Ticker", "Last", "Volume", "IV Rank", "IV Perc"]

        for i in range(0,5):
            table.add_column(col_name[i])

        for i in range(0,11):
            table.add_row(data[i][0], self.last_color(data[i][1]),data[i][2],data[i][3],data[i][4])

        self.console.print(table)

In [12]:
new_printer = Renderer()

In [13]:
new_printer.screener_table(data=etf_print)

                 Screened ETF                  
┏━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Ticker ┃ Last  ┃ Volume ┃ IV Rank ┃ IV Perc ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ KOLD   │ 7.12  │ 4,237  │ 100.00% │ 100%    │
│ BOIL   │ 82.30 │ 2,086  │ 100.00% │ 100%    │
│ UNG    │ 20.16 │ 49,724 │ 77.68%  │ 99%     │
│ FDD    │ 13.90 │ 236    │ 100.00% │ 99%     │
│ DBA    │ 19.12 │ 2,652  │ 68.56%  │ 96%     │
│ FXI    │ 38.93 │ 34,630 │ 58.58%  │ 95%     │
│ JEPI   │ 59.61 │ 149    │ 85.43%  │ 94%     │
│ URA    │ 23.79 │ 2,093  │ 52.59%  │ 94%     │
│ YINN   │ 10.25 │ 3,101  │ 59.59%  │ 93%     │
│ VPN    │ 16.75 │ 165    │ 95.27%  │ 92%     │
│ YANG   │ 19.30 │ 1,013  │ 53.91%  │ 92%     │
└────────┴───────┴────────┴─────────┴─────────┘